In [ ]:
!pip install weaviate-client

## Load data
Download data from [drive](https://drive.google.com/file/d/1W8nBPZA2j1_6AGnw2BAe6ydXLNXzuDq2/view?usp=share_link)

In [2]:
import os
import weaviate

client = weaviate.Client(
    url="https://cohere-wiki-demo.weaviate.network",
    additional_headers={
        "X-Cohere-Api-Key": os.getenv("COHERE_API_KEY")
    }
)
client.is_ready()

True